# Classification using Neural Network by Sex


In [1]:
from pylab import *
import pandas as pd
import random
from sklearn.preprocessing import MinMaxScaler
from sklearn import datasets, linear_model
from sklearn.model_selection import train_test_split
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn import *
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout

%matplotlib inline
#columns = ['BodyweightKg', 'Age', 'Best3SquatKg', 'Best3BenchKg','Best3DeadliftKg']
best_columns = ['Best3SquatKg', 'Best3BenchKg', 'Best3DeadliftKg']
columns = ['BodyweightKg', 'Equipment', 'Age', 'Squat1Kg', 'Squat2Kg', 'Squat3Kg', 'Squat4Kg', 'Best3SquatKg', 'Bench1Kg', 'Bench2Kg', 'Bench3Kg', 'Bench4Kg', 'Best3BenchKg', 'Deadlift1Kg', 'Deadlift2Kg', 'Deadlift3Kg', 'Deadlift4Kg', 'Best3DeadliftKg']
columns_no_best = ['BodyweightKg', 'Equipment', 'Age', 'Squat1Kg', 'Squat2Kg', 'Squat3Kg', 'Squat4Kg', 'Bench1Kg', 'Bench2Kg', 'Bench3Kg', 'Bench4Kg', 'Deadlift1Kg', 'Deadlift2Kg', 'Deadlift3Kg', 'Deadlift4Kg']
lifts_nodemo = ['Equipment', 'Squat1Kg', 'Squat2Kg', 'Squat3Kg', 'Squat4Kg', 'Bench1Kg', 'Bench2Kg', 'Bench3Kg', 'Bench4Kg', 'Deadlift1Kg', 'Deadlift2Kg', 'Deadlift3Kg', 'Deadlift4Kg']
dv = pd.read_csv('data.csv')
print(dv)

Using TensorFlow backend.


                         Name Sex Event Equipment   Age AgeClass  \
0                Abbie Murphy   F   SBD     Wraps  29.0    24-34   
1                 Abbie Tuong   F   SBD     Wraps  29.0    24-34   
2              Ainslee Hooper   F     B       Raw  40.0    40-44   
3             Amy Moldenhauer   F   SBD     Wraps  23.0    20-23   
4                Andrea Rowan   F   SBD     Wraps  45.0    45-49   
...                       ...  ..   ...       ...   ...      ...   
608618          Adam Jasiński   M   SBD       Raw  65.5    65-69   
608619  Włodzimierz Żmijewski   M   SBD       Raw  67.5    65-69   
608620         Marian Cafalik   M   SBD       Raw  60.5    60-64   
608621     Marian Piwowarczyk   M   SBD       Raw  55.5    55-59   
608622     Andrzej Bryniarski   M   SBD       Raw  62.5    60-64   

        BodyweightKg WeightClassKg  Squat1Kg  Squat2Kg  ...  Deadlift3Kg  \
0               59.8            63      80.0      92.5  ...        130.0   
1               58.5           

In [2]:
# Dropping columns not needed for this test
dv = dv.drop(columns=['Name', 'AgeClass', 
                         'Place', 'Tested',
                          'Date', 'McCulloch', 'Glossbrenner',
                          'WeightClassKg'])
dv = dv.where(dv['Event'] == 'SBD') # --> All lifts subset
dv.dropna(subset=['Event'], inplace=True)
print(dv.isnull().sum())

Sex                     0
Event                   0
Equipment               0
Age                     0
BodyweightKg            0
Squat1Kg           169146
Squat2Kg           171653
Squat3Kg           177868
Squat4Kg           393317
Best3SquatKg          177
Bench1Kg           169181
Bench2Kg           171184
Bench3Kg           177819
Bench4Kg           393594
Best3BenchKg          182
Deadlift1Kg        169016
Deadlift2Kg        172429
Deadlift3Kg        182072
Deadlift4Kg        392174
Best3DeadliftKg       166
TotalKg                 0
Wilks                   0
dtype: int64


In [3]:
print(dv[dv['Event'] == 'SBD'].shape)

(395708, 22)


In [4]:
dv.dropna(subset=best_columns, inplace=True)
dv.reset_index(drop=True, inplace=True)
print('Dataset shape' + str(dv.shape))
print('Lifters in Squat, Bench, Deadlift competitions: ' + str(dv['Sex'].count()))
dv.isnull().sum()

Dataset shape(395404, 22)
Lifters in Squat, Bench, Deadlift competitions: 395404


Sex                     0
Event                   0
Equipment               0
Age                     0
BodyweightKg            0
Squat1Kg           169042
Squat2Kg           171544
Squat3Kg           177747
Squat4Kg           393015
Best3SquatKg            0
Bench1Kg           169077
Bench2Kg           171075
Bench3Kg           177693
Bench4Kg           393292
Best3BenchKg            0
Deadlift1Kg        168913
Deadlift2Kg        172299
Deadlift3Kg        181928
Deadlift4Kg        391872
Best3DeadliftKg         0
TotalKg                 0
Wilks                   0
dtype: int64

## Last modifications to use the data for training
Source: https://developers.google.com/machine-learning/data-prep/transform/normalization
First normalisation of data:
- Age: z-score scaling to account for outliers
- BodyweightKg: 

In [5]:
# Convert string data to numeric
numericSex = {'M': 1.0, 'F': 0.0}
# Boolean for equipment
numericEquipment = {'Raw': 0.0, 'Wraps': 1.0, 'Multi-ply': 1.0, 'Single-ply': 1.0, 'Straps': 1.0}

dv.Sex = [numericSex[item] for item in dv.Sex]
dv.Equipment = [numericEquipment[item] for item in dv.Equipment]

dv.apply(lambda x: np.clip(x, a_min=0, a_max=1500.00, inplace=True) if x.name in columns else x)

# Normalise the data, scaling to values between 0-1
scaler = MinMaxScaler(feature_range=(0,1))
dv[columns] = scaler.fit_transform(dv[columns])

In [6]:
# define the target variable (dependent variable) as y
# x = dv['BodyweightKg', 'Equipment', 'Age', 'Squat1Kg', 'Squat2Kg', 'Squat3Kg', 'Squat4Kg', 'Best3SquatKg', 'Bench1Kg', 'Bench2Kg', 'Bench3Kg', 'Bench4Kg', 'Best3BenchKg', 'Deadlift1Kg', 'Deadlift2Kg', 'Deadlift3Kg', 'Deadlift4Kg', 'Best3DeadliftKg']
x = dv.filter(items=['Age', 'BodyweightKg', 'Best3SquatKg', 'Best3BenchKg', 'Best3DeadliftKg'])
#y = dv.filter(items=['Sex'])
y = dv['Sex']
# create training and testing vars
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=3, shuffle=True)
print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)

(296553, 5) (296553,)
(98851, 5) (98851,)


In [7]:


model = Sequential()
model.add(Dense(12, input_dim=5, kernel_initializer='uniform', activation='relu'))
model.add(Dense(8, kernel_initializer='uniform', activation='relu'))
model.add(Dense(1, kernel_initializer='uniform', activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [8]:
model.fit(x_train, y_train, epochs=1, batch_size=10, verbose=1)


Epoch 1/1
296553/296553 [==============================] - 23s 79us/step - loss: 0.2175 - accuracy: 0.9163


In [9]:
results = model.predict(x_test)
round_results = [round(test[0]) for test in results]

successes = 0
for result, value in zip(round_results, y_test):
    if result == value:
        successes+=1
accuracy = abs(round((((len(round_results) - successes)/ len(round_results)) * 100) - 100, 5))

print(successes)
print("Accuracy:", accuracy, "%")

91453
Accuracy: 92.51601 %


Now we knows that running this model on these variables results in a high accuracy but what happens when we introduce variables that include NaN values and that are more irregular in nature. (Different Lifts)

We can handle the missing data by replacing NaN values with zero as those missing values represent someone not lifting something, or we can handle them by dropping values in selected columns and limiting the testing.

In [10]:
dv2 = dv.apply(lambda x: x.fillna(0) if x.name in ['BodyweightKg', 'Equipment', 'Age', 'Squat1Kg', 'Squat2Kg', 'Squat3Kg', 'Squat4Kg', 'Best3SquatKg', 'Bench1Kg', 'Bench2Kg', 'Bench3Kg', 'Bench4Kg', 'Best3BenchKg', 'Deadlift1Kg', 'Deadlift2Kg', 'Deadlift3Kg', 'Deadlift4Kg', 'Best3DeadliftKg'] else x)
x2 = dv2.filter(items=lifts_nodemo)
y2 = dv2['Sex']

x2_train, x2_test, y2_train, y2_test = train_test_split(x2, y2, test_size=0.25, random_state=3, shuffle=True)

In [11]:
model2 = Sequential()
model2.add(Dense(12, input_dim=13, kernel_initializer='uniform', activation='relu'))
model2.add(Dense(8, kernel_initializer='uniform', activation='relu'))
model2.add(Dense(1, kernel_initializer='uniform', activation='sigmoid'))

model2.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])


In [12]:
model2.fit(x2_train, y2_train, epochs=1, batch_size=10, verbose=1)


results2 = model2.predict(x2_test)
round_results2 = [round(test[0]) for test in results2]

successes2 = 0
for result, value in zip(round_results2, y2_test):
    if result == value:
        successes2+=1
accuracy2 = abs(round((((len(round_results2) - successes2)/ len(round_results2)) * 100) - 100, 5))

print("Accurate predictions:", successes2)
print("Accuracy:", accuracy2, "%")

Epoch 1/1
296553/296553 [==============================] - 23s 78us/step - loss: 0.4125 - accuracy: 0.8142
Accurate predictions: 82758
Accuracy: 83.71994 %


In [13]:
pred_equip = ['BodyweightKg', 'Sex', 'Age', 'Squat1Kg', 'Squat2Kg', 'Squat3Kg', 'Squat4Kg', 'Bench1Kg', 'Bench2Kg', 'Bench3Kg', 'Bench4Kg', 'Deadlift1Kg', 'Deadlift2Kg', 'Deadlift3Kg', 'Deadlift4Kg']

dv3 = dv.apply(lambda x: x.fillna(0) if x.name in ['BodyweightKg', 'Equipment', 'Age', 'Squat1Kg', 'Squat2Kg', 'Squat3Kg', 'Squat4Kg', 'Best3SquatKg', 'Bench1Kg', 'Bench2Kg', 'Bench3Kg', 'Bench4Kg', 'Best3BenchKg', 'Deadlift1Kg', 'Deadlift2Kg', 'Deadlift3Kg', 'Deadlift4Kg', 'Best3DeadliftKg'] else x)
#x3 = dv3.filter(items=pred_equip)
x3 = dv3.filter(items=['Best3SquatKg', 'Best3BenchKg', 'Best3DeadliftKg', 'BodyweightKg', 'Sex', 'Age']).astype(float)
y3 = dv3['Equipment'].astype(float)

x3_train, x3_test, y3_train, y3_test = train_test_split(x3, y3, test_size=0.25, random_state=3, shuffle=True)

In [14]:
model3 = Sequential()
model3.add(Dense(6, input_dim=6, kernel_initializer='uniform', activation='relu'))
model3.add(Dense(6, kernel_initializer='uniform', activation='relu'))
model3.add(Dropout(0.5))
model3.add(Dense(1, kernel_initializer='uniform', activation='sigmoid'))

model3.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])


In [15]:
model3.fit(x3_train, y3_train, epochs=10, batch_size=64, verbose=1)


results3 = model3.predict(x3_test)
round_results3 = [round(test[0]) for test in results3]

successes3 = 0
for result, value in zip(round_results3, y3_test):
    if result == value:
        successes3+=1
accuracy3 = abs(round((((len(round_results3) - successes3)/ len(round_results3)) * 100) - 100, 5))

print("Accurate predictions:", successes3)
print("Accuracy:", accuracy3, "%")

Epoch 1/10
296553/296553 [==============================] - 4s 14us/step - loss: 0.6362 - accuracy: 0.6257
Epoch 2/10
296553/296553 [==============================] - 4s 14us/step - loss: 0.5967 - accuracy: 0.6704
Epoch 3/10
296553/296553 [==============================] - 4s 13us/step - loss: 0.5884 - accuracy: 0.6775
Epoch 4/10
296553/296553 [==============================] - 4s 14us/step - loss: 0.5837 - accuracy: 0.6833
Epoch 5/10
296553/296553 [==============================] - 4s 13us/step - loss: 0.5825 - accuracy: 0.6847
Epoch 6/10
296553/296553 [==============================] - 4s 14us/step - loss: 0.5816 - accuracy: 0.6856
Epoch 7/10
296553/296553 [==============================] - 4s 14us/step - loss: 0.5820 - accuracy: 0.6869
Epoch 8/10
296553/296553 [==============================] - 4s 13us/step - loss: 0.5814 - accuracy: 0.6863
Epoch 9/10
296553/296553 [==============================] - 4s 14us/step - loss: 0.5815 - accuracy: 0.6866
Epoch 10/10
296553/296553 [==========

In [16]:
model3.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_7 (Dense)              (None, 6)                 42        
_________________________________________________________________
dense_8 (Dense)              (None, 6)                 42        
_________________________________________________________________
dropout_1 (Dropout)          (None, 6)                 0         
_________________________________________________________________
dense_9 (Dense)              (None, 1)                 7         
Total params: 91
Trainable params: 91
Non-trainable params: 0
_________________________________________________________________


In [19]:
def create_baseline():
    # create model
    model = Sequential()
    model.add(Dense(6, input_dim=6, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    # compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

# evaluate model with standardised dataset
estimator = KerasClassifier(build_fn=create_baseline, epochs=5, batch_size=32, verbose=1)
kfold = StratifiedKFold(n_splits=10, shuffle=True)
results = cross_val_score(estimator, x3, y3, cv=kfold)
print("Baseline: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Epoch 1/5
355863/355863 [==============================] - 8s 23us/step - loss: 0.5970 - accuracy: 0.6706
Epoch 2/5
355863/355863 [==============================] - 8s 23us/step - loss: 0.5530 - accuracy: 0.7143
Epoch 3/5
355863/355863 [==============================] - 8s 23us/step - loss: 0.5450 - accuracy: 0.7195
Epoch 4/5
355863/355863 [==============================] - 8s 22us/step - loss: 0.5407 - accuracy: 0.7211
Epoch 5/5
39541/39541 [==============================] - 0s 9us/step
Epoch 1/5
355863/355863 [==============================] - 8s 23us/step - loss: 0.5788 - accuracy: 0.6899
Epoch 2/5
355863/355863 [==============================] - 8s 22us/step - loss: 0.5426 - accuracy: 0.7180
Epoch 3/5
355863/355863 [==============================] - 8s 22us/step - loss: 0.5375 - accuracy: 0.7215
Epoch 4/5
355863/355863 [==============================] - 8s 22us/step - loss: 0.5357 - accuracy: 0.7225
Epoch 5/5
39541/39541 [==============================] - 0s 9us/step
Epoch 1/5
3558

In [32]:
dt = pd.read_csv('data.csv')

dt.Sex = [numericSex[item] for item in dt.Sex]
dt.Equipment = [numericEquipment[item] for item in dt.Equipment]

dt.isnull().sum()

Name                    0
Sex                     0
Event                   0
Equipment               0
Age                     0
AgeClass                0
BodyweightKg            0
WeightClassKg           0
Squat1Kg           380725
Squat2Kg           383286
Squat3Kg           389599
Squat4Kg           606156
Best3SquatKg       211003
Bench1Kg           255162
Bench2Kg           258522
Bench3Kg           269003
Bench4Kg           602385
Best3BenchKg        38576
Deadlift1Kg        353567
Deadlift2Kg        357594
Deadlift3Kg        368866
Deadlift4Kg        602897
Best3DeadliftKg    162908
TotalKg                 0
Place                   0
Wilks                   0
McCulloch               0
Glossbrenner            0
Tested             213762
Date                    0
dtype: int64

In [67]:
print(dt['Tested'].unique())
dt['Tested'].fillna(0.0, inplace=True)
dt['Tested'].replace('Yes', 1.0, inplace=True)
print(dt['Tested'].unique())
dt0 = dt.apply(lambda x: x.fillna(0) if x.name in ['Sex', 'Wilks', 'BodyweightKg', 'Equipment', 'Age', 'Squat1Kg', 'Squat2Kg', 'Squat3Kg', 'Squat4Kg', 'Best3SquatKg', 'Bench1Kg', 'Bench2Kg', 'Bench3Kg', 'Bench4Kg', 'Best3BenchKg', 'Deadlift1Kg', 'Deadlift2Kg', 'Deadlift3Kg', 'Deadlift4Kg', 'Best3DeadliftKg'] else x)
print(dt.shape)
dt.dropna(subset=['Best3SquatKg', 'Best3BenchKg', 'Best3DeadliftKg'], inplace=True)
print(dt.shape)
dt.describe()

[0. 1.]
[0. 1.]
(395404, 30)
(395404, 30)


,Sex,Equipment,Age,BodyweightKg,Squat1Kg,Squat2Kg,Squat3Kg,Squat4Kg,Best3SquatKg,Bench1Kg,...,Deadlift1Kg,Deadlift2Kg,Deadlift3Kg,Deadlift4Kg,Best3DeadliftKg,TotalKg,Wilks,McCulloch,Glossbrenner,Tested
count,395404.000000,395404.000000,395404.000000,395404.000000,226362.000000,223860.000000,217657.000000,2389.000000,395404.000000,226327.000000,...,226491.000000,223105.000000,213476.000000,3532.000000,395404.000000,395404.000000,395404.000000,395404.000000,395404.000000,395404.000000
mean,0.716293,0.532766,31.554391,85.537338,129.699746,107.614336,41.616507,69.508301,193.443383,93.760071,...,173.652354,139.063935,14.470495,66.581758,209.610023,529.532426,380.504369,393.865102,358.232241,0.685532
std,0.450797,0.498926,11.870779,22.386654,138.564813,169.437516,202.898609,199.221585,72.437580,84.469273,...,99.786043,161.850774,220.241951,203.943365,61.685471,179.539478,82.777003,82.671131,81.902583,0.464304
min,0.000000,0.000000,18.000000,30.100000,-526.170000,-580.000000,-600.500000,-550.000000,-257.500000,-427.500000,...,-405.000000,-445.000000,-587.500000,-461.000000,18.100000,40.800000,37.290000,43.440000,35.880000,0.000000
25%,0.000000,0.000000,22.500000,68.900000,100.000000,87.500000,-165.000000,-120.000000,137.500000,62.500000,...,135.000000,125.000000,-217.500000,-150.125000,160.000000,385.000000,324.150000,336.780000,302.020000,0.000000
50%,1.000000,1.000000,28.000000,82.430000,160.000000,157.500000,122.500000,135.000000,190.000000,107.500000,...,190.000000,187.500000,122.500000,147.500000,215.000000,532.500000,376.220000,389.265000,355.810000,1.000000
75%,1.000000,1.000000,38.000000,99.290000,210.000000,215.000000,202.500000,210.000000,240.000000,142.500000,...,230.000000,237.500000,212.500000,215.115000,255.000000,652.500000,433.700000,448.110000,411.922500,1.000000
max,1.000000,1.000000,95.500000,242.400000,555.000000,566.990000,560.000000,505.500000,575.000000,425.000000,...,425.000000,440.000000,457.500000,400.000000,457.500000,1367.500000,779.380000,804.400000,742.960000,1.000000


In [62]:
x4 = dt0.filter(items=['Sex', 'Wilks', 'BodyweightKg', 'Age', 'Squat1Kg', 'Squat2Kg', 'Squat3Kg', 'Best3SquatKg', 'Bench1Kg', 'Bench2Kg', 'Bench3Kg', 'Best3BenchKg', 'Deadlift1Kg', 'Deadlift2Kg', 'Deadlift3Kg', 'Best3DeadliftKg']).astype(float)
y4 = dt0['Equipment']

In [66]:
def create_baseline():
    # create model
    model = Sequential()
    model.add(Dense(16, input_dim=16, activation='relu'))
    model.add(Dense(8, kernel_initializer='uniform', activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))
    # compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

# evaluate model with standardised dataset
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasClassifier(build_fn=create_baseline, epochs=5, batch_size=128, verbose=1)))
pipeline = Pipeline(estimators)
kfold = StratifiedKFold(n_splits=4, shuffle=True)
results = cross_val_score(pipeline, x4, y4, cv=kfold)
print("Standardized: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Epoch 1/5
296553/296553 [==============================] - 7s 25us/step - loss: 0.5782 - accuracy: 0.7040
Epoch 2/5
296553/296553 [==============================] - 7s 23us/step - loss: 0.5566 - accuracy: 0.7210
Epoch 3/5
296553/296553 [==============================] - 7s 23us/step - loss: 0.5539 - accuracy: 0.7215
Epoch 4/5
296553/296553 [==============================] - 7s 24us/step - loss: 0.5536 - accuracy: 0.7226
Epoch 5/5
98851/98851 [==============================] - 1s 11us/step
Epoch 1/5
296553/296553 [==============================] - 7s 25us/step - loss: 0.5743 - accuracy: 0.7066
Epoch 2/5
296553/296553 [==============================] - 7s 24us/step - loss: 0.5549 - accuracy: 0.7197
Epoch 3/5
296553/296553 [==============================] - 7s 24us/step - loss: 0.5523 - accuracy: 0.7204
Epoch 4/5
296553/296553 [==============================] - 8s 25us/step - loss: 0.5506 - accuracy: 0.7212
Epoch 5/5
98851/98851 [==============================] - 1s 11us/step
Epoch 1/5
29

In [68]:
dt0.apply(lambda x: np.clip(x, a_min=0, a_max=1500.00, inplace=True) if x.name in columns else x)

x4 = dt0.filter(items=['Sex', 'Wilks', 'BodyweightKg', 'Age', 'Squat1Kg', 'Squat2Kg', 'Squat3Kg', 'Best3SquatKg', 'Bench1Kg', 'Bench2Kg', 'Bench3Kg', 'Best3BenchKg', 'Deadlift1Kg', 'Deadlift2Kg', 'Deadlift3Kg', 'Best3DeadliftKg']).astype(float)
y4 = dt0['Equipment']

In [69]:
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasClassifier(build_fn=create_baseline, epochs=5, batch_size=128, verbose=1)))
pipeline = Pipeline(estimators)
kfold = StratifiedKFold(n_splits=4, shuffle=True)
results = cross_val_score(pipeline, x4, y4, cv=kfold)
print("Standardized: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Epoch 1/5
296553/296553 [==============================] - 3s 11us/step - loss: 0.5777 - accuracy: 0.7042
Epoch 2/5
296553/296553 [==============================] - 3s 10us/step - loss: 0.5536 - accuracy: 0.7194
Epoch 3/5
296553/296553 [==============================] - 3s 10us/step - loss: 0.5512 - accuracy: 0.7204 0s - l
Epoch 4/5
296553/296553 [==============================] - 3s 10us/step - loss: 0.5506 - accuracy: 0.7214
Epoch 5/5
98851/98851 [==============================] - 0s 4us/step: 
Epoch 1/5
296553/296553 [==============================] - 3s 11us/step - loss: 0.5652 - accuracy: 0.7133
Epoch 2/5
296553/296553 [==============================] - 3s 11us/step - loss: 0.5471 - accuracy: 0.7293
Epoch 3/5
296553/296553 [==============================] - 4s 13us/step - loss: 0.5440 - accuracy: 0.7306
Epoch 4/5
296553/296553 [==============================] - 3s 11us/step - loss: 0.5436 - accuracy: 0.7309
Epoch 5/5
98851/98851 [==============================] - 0s 5us/step
Epoch

In [70]:
x5 = dt.filter(items=['Sex', 'Wilks', 'BodyweightKg', 'Age', 'Best3SquatKg', 'Best3BenchKg', 'Best3DeadliftKg']).astype(float)
y5 = dt['Equipment']

In [71]:
def create_baseline():
    # create model
    model = Sequential()
    model.add(Dense(7, input_dim=7, activation='relu'))
    model.add(Dense(4, kernel_initializer='uniform', activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))
    # compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

# evaluate model with standardised dataset
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasClassifier(build_fn=create_baseline, epochs=5, batch_size=128, verbose=1)))
pipeline = Pipeline(estimators)
kfold = StratifiedKFold(n_splits=4, shuffle=True)
results = cross_val_score(pipeline, x5, y5, cv=kfold)
print("Standardized: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Epoch 1/5
296553/296553 [==============================] - 3s 11us/step - loss: 0.6322 - accuracy: 0.6713
Epoch 2/5
296553/296553 [==============================] - 3s 10us/step - loss: 0.6181 - accuracy: 0.6735
Epoch 3/5
296553/296553 [==============================] - 3s 10us/step - loss: 0.6179 - accuracy: 0.6745
Epoch 4/5
296553/296553 [==============================] - 3s 11us/step - loss: 0.6171 - accuracy: 0.6756
Epoch 5/5
98851/98851 [==============================] - 0s 4us/step
Epoch 1/5
296553/296553 [==============================] - 3s 11us/step - loss: 0.5984 - accuracy: 0.6592
Epoch 2/5
296553/296553 [==============================] - 3s 10us/step - loss: 0.5814 - accuracy: 0.6748
Epoch 3/5
296553/296553 [==============================] - 3s 11us/step - loss: 0.5796 - accuracy: 0.6767
Epoch 4/5
296553/296553 [==============================] - 3s 10us/step - loss: 0.5781 - accuracy: 0.6772
Epoch 5/5
98851/98851 [==============================] - 0s 4us/step
Epoch 1/5
2965

In [77]:
dt.apply(lambda x: np.clip(x, a_min=0, a_max=1500.00, inplace=True) if x.name in columns else x)

x5 = dt.filter(items=['Tested', 'Sex', 'Wilks', 'BodyweightKg', 'Age', 'Best3SquatKg', 'Best3BenchKg', 'Best3DeadliftKg']).astype(float)
y5 = dt['Equipment']

In [78]:
def create_baseline():
    # create model
    model = Sequential()
    model.add(Dense(8, input_dim=8, activation='relu'))
    model.add(Dense(4, kernel_initializer='uniform', activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))
    # compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasClassifier(build_fn=create_baseline, epochs=5, batch_size=64, verbose=1)))
pipeline = Pipeline(estimators)
kfold = StratifiedKFold(n_splits=2, shuffle=True)
results = cross_val_score(pipeline, x5, y5, cv=kfold)
print("Standardized: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Epoch 1/5
197702/197702 [==============================] - 4s 20us/step - loss: 0.6100 - accuracy: 0.6657
Epoch 2/5
197702/197702 [==============================] - 4s 19us/step - loss: 0.5918 - accuracy: 0.6825
Epoch 3/5
197702/197702 [==============================] - 4s 19us/step - loss: 0.5878 - accuracy: 0.6848
Epoch 4/5
197702/197702 [==============================] - 4s 19us/step - loss: 0.5871 - accuracy: 0.6850
Epoch 5/5
197702/197702 [==============================] - 2s 8us/step
Epoch 1/5
197702/197702 [==============================] - 4s 20us/step - loss: 0.5955 - accuracy: 0.6573
Epoch 2/5
197702/197702 [==============================] - 4s 19us/step - loss: 0.5716 - accuracy: 0.6813
Epoch 3/5
197702/197702 [==============================] - 4s 19us/step - loss: 0.5698 - accuracy: 0.6834
Epoch 4/5
197702/197702 [==============================] - 4s 19us/step - loss: 0.5711 - accuracy: 0.6822
Epoch 5/5
197702/197702 [==============================] - 1s 7us/step
Standardiz

In [81]:
dt0.apply(lambda x: np.clip(x, a_min=0, a_max=1500.00, inplace=True) if x.name in columns else x)

x4 = dt0.filter(items=['Tested', 'Sex', 'Wilks', 'BodyweightKg', 'Age', 'Squat1Kg', 'Squat2Kg', 'Squat3Kg', 'Best3SquatKg', 'Bench1Kg', 'Bench2Kg', 'Bench3Kg', 'Best3BenchKg', 'Deadlift1Kg', 'Deadlift2Kg', 'Deadlift3Kg', 'Best3DeadliftKg']).astype(float)
y4 = dt0['Equipment']


def create_baseline():
    # create model
    model = Sequential()
    model.add(Dense(17, input_dim=17, activation='relu'))
    model.add(Dense(8, kernel_initializer='uniform', activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))
    # compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasClassifier(build_fn=create_baseline, epochs=20, batch_size=128, verbose=1)))
pipeline = Pipeline(estimators)
kfold = StratifiedKFold(n_splits=4, shuffle=True)
results = cross_val_score(pipeline, x4, y4, cv=kfold)
print("Standardized: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Epoch 1/20
296553/296553 [==============================] - 3s 11us/step - loss: 0.5174 - accuracy: 0.7563
Epoch 2/20
296553/296553 [==============================] - 3s 11us/step - loss: 0.4930 - accuracy: 0.7735
Epoch 3/20
296553/296553 [==============================] - 3s 11us/step - loss: 0.4909 - accuracy: 0.7748
Epoch 4/20
296553/296553 [==============================] - 4s 14us/step - loss: 0.4886 - accuracy: 0.7764
Epoch 5/20
296553/296553 [==============================] - 3s 11us/step - loss: 0.4869 - accuracy: 0.7766
Epoch 6/20
296553/296553 [==============================] - 3s 11us/step - loss: 0.4854 - accuracy: 0.7776
Epoch 7/20
296553/296553 [==============================] - 3s 11us/step - loss: 0.4859 - accuracy: 0.7772
Epoch 8/20
296553/296553 [==============================] - 3s 11us/step - loss: 0.4856 - accuracy: 0.7785
Epoch 9/20
296553/296553 [==============================] - 3s 11us/step - loss: 0.4848 - accuracy: 0.7788
Epoch 10/20
296553/296553 [==========

296553/296553 [==============================] - 3s 11us/step - loss: 0.4767 - accuracy: 0.7832
Epoch 15/20
296553/296553 [==============================] - 3s 11us/step - loss: 0.4758 - accuracy: 0.7837
Epoch 16/20
296553/296553 [==============================] - 3s 11us/step - loss: 0.4764 - accuracy: 0.7830
Epoch 17/20
296553/296553 [==============================] - 3s 11us/step - loss: 0.4745 - accuracy: 0.7836
Epoch 18/20
296553/296553 [==============================] - 3s 11us/step - loss: 0.4751 - accuracy: 0.7831 0s - loss: 0.4756 
Epoch 19/20
296553/296553 [==============================] - 3s 11us/step - loss: 0.4752 - accuracy: 0.7830
Epoch 20/20
98851/98851 [==============================] - 0s 5us/step
Standardized: 79.40% (0.14%)


In [90]:
dt['Tested'].isnull()
dx = dt.where(dt['Tested'] == 1) # --> All lifts subset
dx.dropna(subset=['Tested'], inplace=True)
dx.shape

(271062, 30)

In [98]:
dt.where(dt['Place'] == 'G').count()

Name               1205
Sex                1205
Event              1205
Equipment          1205
Age                1205
AgeClass           1205
BodyweightKg       1205
WeightClassKg      1205
Squat1Kg           1000
Squat2Kg            979
Squat3Kg            944
Squat4Kg              2
Best3SquatKg       1205
Bench1Kg            998
Bench2Kg            988
Bench3Kg            957
Bench4Kg              1
Best3BenchKg       1205
Deadlift1Kg        1001
Deadlift2Kg         984
Deadlift3Kg         930
Deadlift4Kg           3
Best3DeadliftKg    1205
TotalKg            1205
Place              1205
Wilks              1205
McCulloch          1205
Glossbrenner       1205
Tested             1205
Date               1205
dtype: int64

IndexError: single positional indexer is out-of-bounds